# ResNet-E2E

## Datasets
按照摄像头类型将数据分为三类，`L`,`M`以及`R`。其中`L`与`R`的"Steering"标签会被人为添加一个偏移量，模拟汽车在偏离车道的情况下反向修正的操作，且这两种数据会全部用于训练模型（被分配到Training Set）。而`M`的数据会根据Training: Eval: Test = 6:3:1的比例来进行划分。


In [20]:
# Import required libraries
import os
import torch
from torch.utils.data import Subset,DataLoader
import torchvision.transforms as transforms
from data_loader import SimulatorDataset  # Ensure data_loader.py is in the same directory
from network_model import ModelCNN  # 
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split


In [96]:
# Simulator Data Loading Example
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set paths to the image and label directories
image_dir = "./data/Garching_dataset/straight/images"  # Replace with your actual image directory
label_dir = "./data/Garching_dataset/straight/drivings"  # Replace with your actual label directory

# Define transformation for images (e.g., resizing and normalization)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),          # Convert PIL images to tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize pixel values to [-1, 1]
])

## Data splitting

In [97]:

# 按摄像头标签（L, M, R）分类数据
def categorize_by_camera(dataset):
    camera_data = {"L": [], "M": [], "R": []}
    for idx in range(len(dataset)):
        image_path = dataset.data_map[idx]["image"]
        camera_type = dataset.data_map[idx]["camera"]
        camera_data[camera_type].append(idx)
    return camera_data


In [98]:
output_fields = ["Steering", "Throttle", "Brake"]  # 你想要的输出字段

# Initialize dataset
dataset = SimulatorDataset(
    image_dir=image_dir,
    label_dir=label_dir,
    transform= transform,
    output_fields=output_fields # Specify desired output fields
)


Found 12948 image files in ./data/Garching_dataset/straight/images.
Found 4316 label files in ./data/Garching_dataset/straight/drivings.


Mapping Images to Labels: 100%|██████████| 12948/12948 [00:00<00:00, 110904.09file/s]

Successfully matched 12948 image-label pairs.
Created data_map with 12948 entries.


In [99]:
# 检验，取出第一个样本
image, label = dataset[0]
print(image.shape)


Loaded image at index 0: ./data/Garching_dataset/straight/images\20241103_164545_000_L.png (Size: (640, 360))
successfully transform the image
Transformed image shape: torch.Size([3, 224, 224])
Parsed labels from ./data/Garching_dataset/straight/drivings\20241103_164545_000.csv: {'Throttle': 1.0, 'Brake': 0.0, 'Steering': 0.0, 'Wheel Angle': 0.0, 'Heading': -0.195734, 'Position': [-26481.492, 13759.909, 4.352], 'Speed': 1.805358, 'Acc': 0.208302, 'Direction': 1}
Output labels for index 0: [0.20000000298023224, 1.0, 0.0]
torch.Size([3, 224, 224])


In [100]:
# 将数据分类为 L, M, R
camera_data = categorize_by_camera(dataset)
data_L = camera_data["L"]
data_M = camera_data["M"]
data_R = camera_data["R"]
print(len(data_L))
print(len(data_M))
print(len(data_R))

Dataset length: 12948
4316
4316
4316


## 大数据集分割

In [116]:

# 对 M 数据划分为 train, eval, test
train_M, temp_M = train_test_split(data_M, test_size=0.25, random_state=42)
eval_M, test_M = train_test_split(temp_M, test_size=0.25, random_state=42)  # 30% eval, 10% test

# train 集合包括 L、R 和部分 M
train_indices = data_L + data_R + train_M

# eval 和 test 集合仅包括 M
eval_indices = eval_M
test_indices = test_M

# 打印划分结果
print(f"Train: {len(train_indices)} samples")
print(f"Eval: {len(eval_indices)} samples")
print(f"Test: {len(test_indices)} samples")

Train: 11869 samples
Eval: 809 samples
Test: 270 samples


## 小数据集分割

In [117]:
print(len(data_L))
print(len(data_M))
print(len(data_R))

4316
4316
4316


In [118]:
import random
# 从每个摄像头类别中抽取 50% 的数据
random.seed(42)  # 固定随机种子，确保可重复性
small_data_L = random.sample(data_L, len(data_L) // 3)
small_data_M = random.sample(data_M, len(data_M) // 3)
small_data_R = random.sample(data_R, len(data_R) // 3)

In [119]:

# 对小数据集的 M 图像进行 6:3:1 分割
train_M, temp_M = train_test_split(small_data_M, test_size=0.25, random_state=42)
eval_M, test_M = train_test_split(temp_M, test_size=0.25, random_state=42)  # 30% eval, 10% test


# 小数据集 train 集合包括 L、R 和部分 M
small_train_indices = small_data_L + small_data_R + train_M

# 小数据集 eval 和 test 集合仅包括 M
small_eval_indices = eval_M
small_test_indices = test_M

# 打印小数据集划分结果
print(f"Small Train: {len(small_train_indices)} samples")
print(f"Small Eval: {len(small_eval_indices)} samples")
print(f"Small Test: {len(small_test_indices)} samples")
    


Small Train: 3954 samples
Small Eval: 270 samples
Small Test: 90 samples


# Use dataloader traininig

In [120]:
# Flag to decide which dataset to use
use_small_dataset = False  # 设置为 True 使用小数据集，False 使用大数据集


In [ ]:
# 超参数
batch_size = 10
num_epochs = 50
learning_rate = 1e-4

In [122]:
# 根据 flag 决定使用大数据集还是小数据集
if use_small_dataset:
    print("Using small dataset for training and evaluation...")
    train_dataset = Subset(dataset, train_indices)
    eval_dataset = Subset(dataset, eval_indices)
    test_dataset = Subset(dataset, test_indices)

    train_loader =  DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    eval_loader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
    test_loader =  DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
else:
    print("Using full dataset for training and evaluation...")
    train_dataset = Subset(dataset, train_indices)
    eval_dataset = Subset(dataset, eval_indices)
    test_dataset = Subset(dataset, test_indices)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
    eval_loader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)


Using full dataset for training and evaluation...


In [123]:
# 初始化模型
model = ModelCNN(output_fields=output_fields)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [124]:
# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

# 使用当前时间戳生成独特的日志目录名称
# 超参数和自定义消息
custom_message = "straight_big_1st"

# 动态生成日志目录
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_dir = f"./logs/lr_{learning_rate}_msg_{custom_message}_{current_time}"


# 定义模型保存路径
save_dir = "./models"
os.makedirs(save_dir, exist_ok=True)  # 如果目录不存在，则创建



In [ ]:
# 初始化 TensorBoard
writer = SummaryWriter(log_dir=log_dir)
print(f"TensorBoard logs will be saved in: {log_dir}")

for epoch in range(num_epochs):
    # 训练阶段
    model.train()
    train_loss = 0

    # tqdm 进度条 for 训练阶段
    with tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs} - Training", leave=False) as train_bar:
        for images, labels in train_bar:
            images, labels = images.to(device), labels.to(device)

            # 前向传播
            outputs = model(images)
            loss = criterion(outputs, labels)

            # 反向传播和优化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            # 更新 tqdm 信息
            train_bar.set_postfix({"Batch Loss": f"{loss.item():.4f}"})

    # 计算 epoch 平均训练损失
    avg_train_loss = train_loss / len(train_loader)

    # 验证阶段
    model.eval()
    eval_loss = 0

    # tqdm 进度条 for 验证阶段
    with tqdm(eval_loader, desc=f"Epoch {epoch + 1}/{num_epochs} - Validating", leave=False) as eval_bar:
        with torch.no_grad():
            for images, labels in eval_bar:
                images, labels = images.to(device), labels.to(device)

                # 前向传播
                outputs = model(images)
                loss = criterion(outputs, labels)
                eval_loss += loss.item()

                # 更新 tqdm 信息
                eval_bar.set_postfix({"Batch Loss": f"{loss.item():.4f}"})

    # 计算 epoch 平均验证损失
    avg_eval_loss = eval_loss / len(eval_loader)

    # 打印结果
    print(f"Epoch [{epoch + 1}/{num_epochs}]")
    print(f"Train Loss: {avg_train_loss:.4f}")
    print(f"Eval Loss: {avg_eval_loss:.4f}")

    # 在同一张 TensorBoard 图上记录训练和验证损失
    writer.add_scalars("Loss", {"Train": avg_train_loss, "Eval": avg_eval_loss}, epoch)

    # 可选：记录网络权重的直方图到 TensorBoard
    for name, param in model.named_parameters():
        writer.add_histogram(name, param, epoch)

    # **保存断点**
    if (epoch + 1) % 10 == 0:  # 每 10 个 epochs 保存一次模型
        checkpoint_path = os.path.join(save_dir, f"checkpoint_epoch_{custom_message}_{epoch + 1}.pth")
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_train_loss,
        }, checkpoint_path)
        print(f"Checkpoint saved to {checkpoint_path}")

# 关闭 TensorBoard writer
writer.close()

# 定义模型保存路径
final_model_path = os.path.join(save_dir, "model_straight_1_3outputs.pth")
torch.save(model.state_dict(), final_model_path)
print(f"Final model saved to {final_model_path}")


TensorBoard logs will be saved in: ./logs/lr_0.0001_msg_straight_big_2024-11-20_01-01-45


Epoch [1/15]
Train Loss: 0.3025
Eval Loss: 0.2013


Epoch [2/15]
Train Loss: 0.2254
Eval Loss: 0.1696


KeyboardInterrupt: 

In [ ]:
# 测试阶段
test_loss = 0
model.eval()

# tqdm 进度条 for 测试阶段
with tqdm(test_loader, desc="Testing", leave=False) as test_bar:
    with torch.no_grad():
        for images, labels in test_bar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

# 计算测试集平均损失
avg_test_loss = test_loss / len(test_loader)
print(f"Test Loss: {avg_test_loss:.4f}")


Test Loss: 0.0903


Model saved to ./models\model_right_1st.pth
